In [1]:
import os
import pickle
import json
from tqdm import tqdm


In [2]:
from collections import defaultdict


In [3]:
os.environ["CODE"] = "/home/aarslan/mq"
os.environ["SCRATCH"] = "/data/aarslan"


In [4]:
with open(os.path.join(os.environ["SCRATCH"], "ego4d_data/v2/analysis_data/ground_truth_labels/ground_truth_labels.pickle"), "rb") as reader:
    ground_truth_labels = pickle.load(reader)


In [5]:
with open(os.path.join(os.environ["CODE"], "scripts/06_analyze_frame_features/02_map_label_dependency_parsing_features_and_blip2_answer_dependency_parsing_features", "label_verb_noun_tool_mapping.json"), "r") as reader:
    label_verb_noun_tools_mapping = json.load(reader)


OSError: [Errno 127] Key has expired: '/home/aarslan/mq/scripts/06_analyze_frame_features/02_map_label_dependency_parsing_features_and_blip2_answer_dependency_parsing_features/label_verb_noun_tool_mapping.json'

In [ ]:
label_verb_noun_tools_mapping_keys = sorted(list(label_verb_noun_tools_mapping.keys())) + ["background"]

label_index_label_mapping = dict()
for i in range(len(label_verb_noun_tools_mapping_keys)):
    label_index_label_mapping[i] = label_verb_noun_tools_mapping_keys[i]


In [ ]:
def get_per_frame_match_counts(clip_id, frame_id, question_index):
    current_label_indices = ground_truth_labels[clip_id][frame_id]
    current_labels = [label_index_label_mapping[label_index] for label_index in current_label_indices]
    with open(os.path.join(os.environ["SCRATCH"], f"ego4d_data/v2/analysis_data/dependency_parsing_results/{clip_id}.pickle"), "rb") as reader:
        dependency_parsing_results = reader.read()
    current_blip2_verb_noun_tools = [int(frame_id // 6 * 6)][question_index][1]
    results = {
        "background_match_count": 0,
        "background_count": 0,
        "verb_noun_tool_match_count": 0,
        "verb_noun_match_count": 0,
        "verb_tool_match_count": 0,
        "verb_match_count": 0,
        "noun_match_count": 0,
        "nonbackground_count": 0
    }
    if current_labels[0] == "background":
        results["background_count"] = 1
        if len(current_blip2_verb_noun_tools) == 0:
            results["background_match_count"] = 1
    else:
        results["nonbackground_count"] = len(current_labels)

        for current_label in current_labels:
            current_label_verb_noun_tool_list = label_verb_noun_tools_mapping[current_label]
            current_verb_noun_tool_match = 0
            current_verb_noun_match = 0
            current_verb_tool_match = 0
            current_verb_match = 0
            current_noun_match = 0
            
            for current_label_verb_noun_tool in current_label_verb_noun_tool_list:
                current_label_verb = current_label_verb_noun_tool[0]
                current_label_noun = current_label_verb_noun_tool[1]
                current_label_tool = current_label_verb_noun_tool[2]
            
                for current_blip2_verb_noun_tool in current_blip2_verb_noun_tools:
                    current_blip2_verb = current_blip2_verb_noun_tool[0]
                    current_blip2_noun = current_blip2_verb_noun_tool[1]
                    current_blip2_tool = current_blip2_verb_noun_tool[2]
                    if current_blip2_verb == current_label_verb and current_blip2_noun == current_label_noun and current_blip2_tool == current_label_tool:
                        current_verb_noun_tool_match = 1
                    if current_blip2_verb == current_label_verb and current_blip2_noun == current_label_noun:
                        current_verb_noun_match = 1
                    if current_blip2_verb == current_label_verb and current_blip2_tool == current_label_tool:
                        current_verb_tool_match = 1
                    if current_blip2_verb == current_label_verb:
                        current_verb_match = 1
                    if current_blip2_noun == current_label_noun:
                        current_noun_match = 1

            results["verb_noun_tool_match_count"] += current_verb_noun_tool_match
            results["verb_noun_match_count"] += current_verb_noun_match
            results["verb_tool_match_count"] += current_verb_tool_match
            results["verb_match_count"] += current_verb_match
            results["noun_match_count"] += current_noun_match

    return results



In [ ]:
question_index_ratios_mapping = dict()
for question_index in [0, 1, 2]:
    question_index_ratios_mapping[question_index] = dict()
    final_match_counts = {
        "background_match_count": 0,
        "background_count": 0,
        "verb_noun_tool_match_count": 0,
        "verb_noun_match_count": 0,
        "verb_tool_match_count": 0,
        "verb_match_count": 0,
        "noun_match_count": 0,
        "nonbackground_count": 0
    }
    for clip_id in tqdm(ground_truth_labels.keys()):
        for frame_id in ground_truth_labels[clip_id].keys():
            current_match_counts = get_per_frame_match_counts(clip_id=clip_id, frame_id=frame_id, question_index=question_index)
            for key, value in current_match_counts.items():
                final_match_counts[key] += value

    question_index_ratios_mapping[question_index]["background_match_ratio"] = final_match_counts["background_match_count"] / float(final_match_counts["background_count"])
    question_index_ratios_mapping[question_index]["verb_noun_tool_match_ratio"] = final_match_counts["verb_noun_tool_match_count"] / float(final_match_counts["nonbackground_count"])
    question_index_ratios_mapping[question_index]["verb_noun_match_ratio"] = final_match_counts["verb_noun_match_count"] / float(final_match_counts["nonbackground_count"])
    question_index_ratios_mapping[question_index]["verb_tool_match_ratio"] = final_match_counts["verb_tool_match_count"] / float(final_match_counts["nonbackground_count"])
    question_index_ratios_mapping[question_index]["verb_match_ratio"] = final_match_counts["verb_match_count"] / float(final_match_counts["nonbackground_count"])
    question_index_ratios_mapping[question_index]["noun_match_ratio"] = final_match_counts["noun_match_count"] / float(final_match_counts["nonbackground_count"])
    


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2495/2495 [01:51<00:00, 22.37it/s]


# Statistics for "What does the image describe?"

In [ ]:
question_index_ratios_mapping[0]


{'background_match_ratio': 0.008209721572784125,
 'verb_noun_tool_match_ratio': 0.02541627356823526,
 'verb_noun_match_ratio': 0.034570419987181536,
 'verb_tool_match_ratio': 0.06758627829675036,
 'verb_match_ratio': 0.10084356989743123,
 'noun_match_ratio': 0.13998811627259145}

# Statistics for "What is the person in this picture doing?"

In [ ]:
question_index_ratios_mapping[1]


{'background_match_ratio': 0.0007187132329506736,
 'verb_noun_tool_match_ratio': 0.026378173804026316,
 'verb_noun_match_ratio': 0.03892561608455127,
 'verb_tool_match_ratio': 0.16319105900562866,
 'verb_match_ratio': 0.20076216755989704,
 'noun_match_ratio': 0.11573235458744478}

# Statistics for "What is happening in this picture?"

In [ ]:
question_index_ratios_mapping[2]

{'background_match_ratio': 0.008207656304873348,
 'verb_noun_tool_match_ratio': 0.034494668128345084,
 'verb_noun_match_ratio': 0.05857382659525769,
 'verb_tool_match_ratio': 0.1190730610192763,
 'verb_match_ratio': 0.1828044022298413,
 'noun_match_ratio': 0.15462541138555383}